In [24]:
import pandas as pd
import numpy as np
import pylab as plt
import os
import sys
sys.path.append('../')
from seislab.src import segyrw
from seislab.src import plot_seismic
from seislab.src import plot_maps
# from seislab.src import keras_models as km
from seislab.src import fbpick as ut
from sklearn.linear_model import LinearRegression
import seaborn as sns
import SegRead as s

In [25]:
filename = 'w1_cdp.sgy'

In [26]:
def Ampl_rms(head,data,t1,t2):
    head['Ampl_RMS'] = 0.0
    for z in head['CDP'].unique():
        ind = head[head['CDP'] == z].index
        T0 = head.iloc[ind]['T0'].reset_index(drop=True)
        int1 = T0*1e-3 - t1
        int2 = T0*1e-3 + t2
        data_temp = data[int(int1[0]/dt):int(int2[0]/dt),ind]
        for i in range(len(data_temp[0,:])):
            head['Ampl_RMS'][ind[i]] = np.sqrt(sum(data_temp[:,i]*data_temp[:,i])/len(data_temp[:,i]))
    return head

def read_segy_head_cdp_off(filename):
    data = segyrw.read_segy_file_obspy(filename)
    data = data.T
    head = segyrw.read_header_segyio_full(filename)
    return head
    dt = segyrw.read_bin_header(filename)
    dt = dt['SampleIntervalReel']*1e-6
    head = head.loc[:,['ReceiverGroupElevation','SourceSurfaceElevation','CDP',
                   'SRCX','SRCY','GRPX','GRPY','CDP_X','CDP_Y','OFFSET']]
    head = head.rename(columns = {'ReceiverGroupElevation':'GRPZ','SourceSurfaceElevation':'SRCZ',
                              'CDP_X':'CDPX','CDP_Y':'CDPY'})
    head = head.sort_values(['CDP', 'OFFSET'])
    data = data[:,head.index]
    head = head.reset_index(drop=True)
    
    return head, data, dt

def metrics_theor_real_data(head, ampl, offsets, interval1, interval2, threshold):   
    head_tmp = head.copy()
    head_tmp = head_tmp[interval1:interval2]
    real = head_tmp['Ampl_RMS'].values[interval1:interval2]/max(head_tmp['Ampl_RMS'].values[interval1:interval2])
    theor = ampl[interval1:interval2]/max(ampl[interval1:interval2])

    diff = abs(theor-real)
    head_tmp['diff_%'] = diff*100
    disp = np.std(diff)*np.std(diff)
    kmean = np.mean(diff)
   
    head_tmp['colors'] = 'g'
    head_tmp['colors'][head_tmp[(abs( head_tmp['diff_%']) >= threshold)].index] = 'r'
   
    
    sns.set()
    fig = plt.figure(figsize = (10,5))
    plt.plot(offsets[interval1:interval2],real,'b.', label = 'Ампл. по сейсм.')
    plt.plot(offsets[interval1:interval2],theor,'g.',label = 'Теорет.распр.')
    plt.xlabel('Offset, m')
    plt.ylabel('norm(max_Ampl)')
    plt.legend()
    plt.title('mean = '+str(np.round(kmean*10000)/10000)+', disp = '+str(np.round(disp*10000)/10000))
    return head_tmp

def plot_metrics(head, offsets, threshold):
    sns.set()
    fig = plt.figure(figsize = (10,5))
    plt.scatter(offsets,head['diff_%'].values,c = head['colors'],s = 10)
    plt.ylim(-100,100)
    plt.title('Разница между теоретическим распр. и реальными данными, пороговое значение = '+str(threshold)+'%')
    plt.xlabel('Offset, m')
    plt.ylabel('diff Ampl %')
    return 

In [27]:
filename1 = filename#directory+filename
head1 = read_segy_head_cdp_off(filename1)
    

ss = s.Seg.SegReader()
ss.open(filename1)
data2, head2=ss.get_data_and_trace_heads()

In [18]:
head1

,SEQWL,CDP_TRACE,CDP_X,CDP_Y,INLINE,XLINE,IDX
0,57939,38923,499950,6637576,808,977,0
1,57940,38923,499950,6637576,808,977,1
2,57941,38923,499950,6637576,808,977,2
3,57942,38923,499950,6637576,808,977,3
4,57943,38923,499950,6637576,808,977,4
...,...,...,...,...,...,...,...
3520,107,45287,500050,6637676,812,981,3520
3521,108,45287,500050,6637676,812,981,3521
3522,109,45287,500050,6637676,812,981,3522
3523,110,45287,500050,6637676,812,981,3523


In [28]:
head2

,TRACE_SEQUENCE_LINE,TRACE_SEQUENCE_FILE,FieldRecord,TraceNumber,EnergySourcePoint,CDP,CDP_TRACE,TraceIdentificationCode,NSummedTraces,NStackedTraces,...,OverTravel,CDP_X,CDP_Y,ILINE_NO,XLINE_NO,ShortpointNumber,ScalarValueForShortpointNumber,TraceValueMeasurementUnit,TransductionConstant,spare
0,57939,0,0,0,0,0,38923,-26613,1,0,...,0,499950,6637576,808,977,0,0,0,0,3633830213348678286034854769904592165554154454...
1,57940,0,0,0,0,0,38923,-26613,1,0,...,0,499950,6637576,808,977,0,0,0,0,3633830213348678286034854769904592165554154454...
2,57941,0,0,0,0,0,38923,-26613,1,0,...,0,499950,6637576,808,977,0,0,0,0,3633830213348678286034854769904592165554154454...
3,57942,0,0,0,0,0,38923,-26613,1,0,...,0,499950,6637576,808,977,0,0,0,0,3633830213348678286034854769904592165554154454...
4,57943,0,0,0,0,0,38923,-26613,1,0,...,0,499950,6637576,808,977,0,0,0,0,3633830213348678286034854769904592165554154454...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3520,107,0,0,0,0,0,45287,-20249,1,0,...,0,500050,6637676,812,981,0,0,0,0,3634557052073143123253415973434060083146311534...
3521,108,0,0,0,0,0,45287,-20249,1,0,...,0,500050,6637676,812,981,0,0,0,0,3634557052073143123253415973434060083146311534...
3522,109,0,0,0,0,0,45287,-20249,1,0,...,0,500050,6637676,812,981,0,0,0,0,3634557052073143123253415973434060083146311534...
3523,110,0,0,0,0,0,45287,-20249,1,0,...,0,500050,6637676,812,981,0,0,0,0,3634557052073143123253415973434060083146311534...


In [ ]:
w